# Personalized Movie Recommendations: A Collaborative Filtering Approach

## 1. Business Understanding
### (a) Introduction

CineCollab, established in 2006, has been at the forefront of digital entertainment, captivating audiences worldwide with a rich collection of films and sophisticated recommendation systems. Our success is deeply rooted in our commitment to elevating user experiences through advanced analytics.

This project mirrors CineCollab's dedication, with a focus on refining recommendation systems. We develop complex predictive models, taking into account factors such as individual viewing habits and user behaviors. By mapping these variables, we intend to work closely with CineCollab to boost the precision and effectiveness of its recommendation algorithm.

The MovieLens dataset plays a pivotal role in realizing this objective. Users can rate movies, and our model harnesses these ratings to craft personalized recommendations. The hurdle lies in designing a system that accurately deciphers user ratings and converts them into relevant film suggestions.

The core business problem we tackle is optimizing user engagement by delivering desired movie recommendations. This involves understanding user preferences based on ratings and creating an intuitive and user-friendly platform for users to provide these ratings. We attempt to maintain a balance between simplicity and capturing nuanced user preferences to ensure the recommendations are both accurate and well-received.

In essence, our project seeks to transform the way users rate movies and receive recommendations, aligning with CineCollab's ongoing commitment to personalized content discovery. Through our model, we aim to not only enhance efficiency but also provide a seamless and enriching experience for users globally, delivering top-notch movie recommendations tailored to their unique tastes and preferences.

### (b) Problem Statement

Online streaming platforms are the dominant form of media consumption but viewers often face a paradox of choice when presented with the entire catalogue. Users struggle to navigate through massive catalogues, leading to subscriber frustration and an increased likelihood of churn. By leveraging the MovieLens dataset comprised of over 100,000 ratings applied to nearly 10,000 films by hundreds of users, we intend to push CineCollabs recommendation algorithm forward. Enhancing recommendations based upon movies streamers prefer has the potential to improve CineCollab's user experience and to help audiences discover unexplored content that matches their interests. This project, implemented successfully will  promote media discovery, encourage niche, cult-like viewers and inspire artistry through expanded access to cinema. Building a working, accurate recommendation system will be the key to unlocking the full creative and commercial potential of CineCollab.

### (c) Defining Metrics of Success

The success of a movie recommendation model using collaborative filtering can be assessed using various metrics that measure its effectiveness in providing accurate and relevant movie suggestions. The combination of the metrics provides a comprehensive understanding of its performance in terms of accuracy, relevance, and user satisfaction. It's essential to choose metrics that align with the specific goals and objectives of the recommendation system and the preferences of the user base.

### (d) Research Questions

1. What features contribute most to the accuracy of collaborative filtering in generating top  movie recommendations?

2. How does the frequency of user ratings influence the accuracy and stability of the movie recommendation model?

3. What are the correlation between user ratings and various movie features?

4. Which movie features demonstrate the highest correlation with collaborative filtering recommendations, and how do they impact the model's predictions?

5. How successful is the collaborative filtering model in providing accurate and tailored movie recommendations based on user ratings and preferences?

### (e) The Main Objective

To develop and implement a movie recommendation system that leverages collaborative filtering techniques to provide personalized top 5 movie recommendations for users.

### (f) The Specific Objectives

1. To clean and preprocess the MovieLens datasets to ensure it is suitable for building a recommendation system.

2. To understand the distribution of movie ratings, explore user behavior, and identify patterns in the datasets.

3. To investigate and compare collaborative filtering techniques for building the recommendation system, such as Singular Value Decomposition (SVD), user-based and item-based.

4. To implement and evaluate the performance of the collaborative filtering model using appropriate metrics such as RSME and MSE.

5. To generate top 5 movie recommendations for a user based on their historical ratings.

### (g) Data Understanding

The MovieLens dataset (ml-latest-small) provides a comprehensive snapshot of user preferences and interactions within the MovieLens movie recommendation service. This dataset, created by 610 users over a period spanning from March 29, 1996, to September 24, 2018, is a valuable resource for gaining insights into user behavior, preferences, and movie metadata.
It has 9 attributes.

**Attributes:**

1. **movieId:** Identifier for movies used by MovieLens.
2. **title:** Contains the names of individual movies and serves as a unique identifier for each film within the dataset.
3. **genres:** Classifies films based on overarching themes, narrative structures, and intended emotional impact.
4. **imdbId:** Identifier associated with a movie on the IMDb (Internet Movie Database) platform.
5. **tmdbId:** Identifier associated with movies on TMDb (The Movie Database).
6. **rating:** Numerical evaluation given by users on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).
7. **timestamp:** represents seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.
8. **tags:** User-generated metadata about movies. Each tag is typically a single word or short phrase, with meaning, value, and purpose determined by each user.
9. **userId:** Unique identifier assigned to each user who participated in movie rating and tagging activities.


**Dataset Statistics:**

100,836 ratings and 3,683 tag applications.
9,742 movies encompassing a diverse array of genres.

